In [1]:
import pandas as pd
from keplergl import KeplerGl
import numpy as np

In [2]:
df = pd.read_csv('CitiBike_with_weather.csv')

C:\Users\anika\AppData\Local\Temp\ipykernel_14980\2820563072.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('CitiBike_with_weather.csv')


In [3]:
# Add a 'value' column for counting trips
df['value'] = 1

In [4]:
# Aggregate trips by start and end stations
df_group = df.groupby(['start_station_name', 'end_station_name'])['value'].count().reset_index()
df_group.rename(columns={'value': 'trips'}, inplace=True)

In [8]:
# Verify the aggregation
print(df_group['trips'].sum())  
print(df.shape)

29838806
(29838806, 17)


In [10]:
df_group.head()


,start_station_name,end_station_name,trips
0,1 Ave & E 110 St,1 Ave & E 110 St,791
1,1 Ave & E 110 St,1 Ave & E 18 St,2
2,1 Ave & E 110 St,1 Ave & E 30 St,4
3,1 Ave & E 110 St,1 Ave & E 39 St,1
4,1 Ave & E 110 St,1 Ave & E 44 St,12


In [11]:
df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'date', 'avgWeather', '_merge', 'value'],
      dtype='object')

In [12]:
# Drop duplicates to get unique station coordinates
df_unique = df.drop_duplicates(subset=['start_station_name', 'end_station_name'])[
    ['start_station_name', 'end_station_name', 'start_lat', 
     'start_lng', 'end_lat', 'end_lng']]

In [13]:
# Merge coordinates back
df_final = df_group.merge(df_unique, on=['start_station_name', 'end_station_name'], how='left')

In [14]:
# Select top 300 routes for performance
df_final = df_final.nlargest(300, 'trips')

In [15]:
# Save filtered data
df_final.to_csv('nyc_bike_data_top_300.csv', index=False)

In [23]:
##Filtered data was used to make the map on Kepler website as the full dataset was too large to load.

In [21]:
# Customization
#Start and End Points: Displayed as blue points with a radius of 5. Blue provides strong visibility against the dark basemap while keeping the focus on clusters of activity.
#Arcs: I used orange for the arc source (where trips start) and blue for the arc target (where trips end). This palette helps distinguish directionality: orange (warmer color) indicates the start, while blue (cooler color) indicates the destination. This makes the flow of trips immediately clear.

In [22]:
# Observations after filtering
#Dense concentrations around Midtown Manhattan – especially near Times Square, Penn Station, and Grand Central.
#Lower Manhattan (Wall Street, Financial District) which stands out as a major work destination.
#Connections to Brooklyn and Queens, which reflect commuting patterns.
#Airport-linked trips (especially LaGuardia), which are prominent because of constant passenger turnover.